<a href="https://colab.research.google.com/github/toaruman/hatamei/blob/main/project2_Kazuki_Ikenoue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import files
from tabulate import tabulate

# ファイルのアップロード
uploaded = files.upload()

# データの読み込み（アップロードされたファイル名を指定してください）
file_path = list(uploaded.keys())[0]
data = pd.read_stata(file_path)

# 画像に基づいてフィルタリングするOECD加盟国
selected_oecd_countries = ["AUS", "AUT", "BEL", "CAN", "DNK", "FIN", "FRA", "DEU", "GRC", "ISL", "IRL", "ITA", "JPN", "NLD", "NZL", "NOR", "PRT", "ESP", "SWE", "CHE", "GBR", "USA"]

# フィルタリング
data_oecd = data[data['countrycode'].isin(selected_oecd_countries)]

# 1990年と2019年のデータにフィルタリング
data_1990 = data_oecd[data_oecd['year'] == 1990]
data_2019 = data_oecd[data_oecd['year'] == 2019]

# 必要な列の抽出
columns_needed = ['countrycode', 'rgdpo', 'emp', 'rkna', 'ctfp', 'labsh']
data_1990 = data_1990[columns_needed]
data_2019 = data_2019[columns_needed]

# 列名に年を付加してマージ
data_1990 = data_1990.add_suffix('_1990')
data_2019 = data_2019.add_suffix('_2019')
data_1990 = data_1990.rename(columns={'countrycode_1990': 'countrycode'})
data_2019 = data_2019.rename(columns={'countrycode_2019': 'countrycode'})

# データのマージ
merged_data = pd.merge(data_1990, data_2019, on='countrycode')

# 成長率の計算
merged_data['Y/L_1990'] = merged_data['rgdpo_1990'] / merged_data['emp_1990']  # 一人当たりGDP 1990
merged_data['Y/L_2019'] = merged_data['rgdpo_2019'] / merged_data['emp_2019']  # 一人当たりGDP 2019
merged_data['K/L_1990'] = merged_data['rkna_1990'] / merged_data['emp_1990']  # 一人当たり資本 1990
merged_data['K/L_2019'] = merged_data['rkna_2019'] / merged_data['emp_2019']  # 一人当たり資本 2019

# 労働生産性の成長率 (Growth Rate)
merged_data['Growth Rate'] = ((merged_data['Y/L_2019'] / merged_data['Y/L_1990'])**(1/29) - 1) * 100

# 資本深化の成長率 (Capital Deepening)
merged_data['Capital Deepening'] = ((merged_data['K/L_2019'] / merged_data['K/L_1990'])**(1/29) - 1) * 100

# 資本分配率（α）の計算
alpha = (merged_data['labsh_1990'] + merged_data['labsh_2019']) / 2

# TFPの成長率 (TFP Growth)
merged_data['TFP Growth'] = merged_data['Growth Rate'] - (alpha * merged_data['Capital Deepening'])

# 資本分配率とTFPのシェアの計算（1990年と2019年の平均）
merged_data['Capital Share'] = alpha
merged_data['TFP Share'] = 1 - alpha

# 必要な列の最終的な抽出
final_columns = ['countrycode', 'Growth Rate', 'TFP Growth', 'Capital Deepening', 'TFP Share', 'Capital Share']
final_data = merged_data[final_columns]

# 選択されたOECD加盟国にフィルタリング
final_data_selected_oecd = final_data[final_data['countrycode'].isin(selected_oecd_countries)]

# 結果の表示
print(tabulate(final_data_selected_oecd, headers='keys', tablefmt='grid'))


Saving pwt1001.dta to pwt1001.dta
+----+---------------+---------------+--------------+---------------------+-------------+-----------------+
|    | countrycode   |   Growth Rate |   TFP Growth |   Capital Deepening |   TFP Share |   Capital Share |
+====+===============+===============+==============+=====================+=============+=================+
|  0 | AUS           |      1.71931  |    0.577143  |             1.90734 |    0.401173 |        0.598827 |
+----+---------------+---------------+--------------+---------------------+-------------+-----------------+
|  1 | AUT           |      2.08297  |    0.899628  |             1.94497 |    0.39159  |        0.60841  |
+----+---------------+---------------+--------------+---------------------+-------------+-----------------+
|  2 | BEL           |      1.53019  |    0.25317   |             2.10429 |    0.393134 |        0.606866 |
+----+---------------+---------------+--------------+---------------------+-------------+-------------